In [12]:
import sys
sys.path.append('..')
from db1 import *
import pandas as pd

R = PolynomialRing(ZZ, 't')
t = R.gen()

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
    """
    if n <= 8 and M.is_connected():
        for i in range(len(mat[n][k])):
            if mat[n][k][i].is_isomorphic(M):
                return ikl[n][k][i]
    """
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * kazhdan_lusztig_uniform_matroid(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_uniform_matroid(k, n) + kazhdan_lusztig_uniform_matroid(k-c+1, h) * kazhdan_lusztig_uniform_matroid(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

First, we initialize the matroid and obtain the lattice of flats. We relabel the flats for simple presentation.

In [13]:
r, n = 5, 6

R.<q> = PolynomialRing(QQ)
matroid = matroids.Uniform(r, n)
flats = list(matroid.lattice_of_flats())

In [14]:
#plottting the geometric lattice:
#size = 0 for now

labels = {element: idx for idx, element in enumerate(flats)}
matroid.lattice_of_flats().plot(
    element_labels = labels, element_color = "white",
    figsize= 0, cover_color = "grey")

Graphics object consisting of 230 graphics primitives

In [15]:
def invKL(M):
	R = PolynomialRing(QQ, 'q')
	q = R.gen(0)
	if M.rank() == 0:
		return R(1)
	L = M.lattice_of_flats()
	invKLpoly = 0
	for i in range(1,len(L)):
		Mcon = M.contract(L[i])
		Mres = M.delete(M.groundset()-L[i])
		Lres = Mres.lattice_of_flats()
		invKLpoly += (-1)^(Mres.rank())*Lres.kazhdan_lusztig_polynomial()*invKL(Mcon)
	return -invKLpoly

invKL(matroids.named_matroids.Vamos())

49*q + 30

Guess 1:
$$Q_M(t) = Q_{M\setminus e}(t)-tQ_{M/e}(t)+\sum_{F\in S}{\tau(M/{F\cup e})t^{\text{crk}(F)/2}Q_{M\mid F}(t)}.$$

Functions needed for the r.h.s. will be written below:

In [16]:
#the right hand side of the equality

def q_tau(matroid, S, e):
    q_tau = [] 
    for F in S:
        modded_matroid = matroid.contract(F.union(e))
        if modded_matroid.rank() % 2 == 0:
            q_tau.append(0)
        else:
            q_tau.append(invKL(modded_matroid).leading_coefficient())
    return q_tau

def tau(matroid, S, e):
    tau = [] 
    for F in S:
        modded_matroid = matroid.contract(F.union(e))
        if modded_matroid.rank() % 2 == 0:
            tau.append(0)
        else:
            tau.append(modded_matroid.lattice_of_flats().kazhdan_lusztig_polynomial().leading_coefficient())
    return tau

def the_set_S(flats, e):
    return set(F for F in flats if e.isdisjoint(F) and F.union(e) in flats)

def delform_rhs(matroid, tau_func, inv_kl):
    e = matroid.flats(1)[0]
    flats = list(matroid.lattice_of_flats())

    deletion = matroid.delete(e).simplify()
    contraction = matroid.contract(e).simplify()

    set_S = list(the_set_S(flats, e))

    # Compute bv_tau_function and zip with set_S
    tau_values = tau_func(matroid, set_S, e)

    # Compute the summation over S
    sum_S = sum(
        x * q ** ((matroid.rank() - matroid.rank(y))//2) * inv_kl(matroid.delete(matroid.groundset() - y).simplify())
        for x, y in zip(tau_values, set_S)
    )
    # Compute the RHS
    return inv_kl(deletion) - (q + 1) * inv_kl(contraction) + sum_S

In [ ]:
# Create a list of matroids
matroids_list = [matroids.Uniform(r, n) for r, n in [(2, 3), (2, 4), (2,5), (3, 4), (4, 5), (5,6), (3, 5), (4, 6)]] + \
    [matroids.Wheel(i) for i in range(3, 6)] + \
    [matroids.Whirl(i) for i in range(3, 5)]

# Create a dataframe to store the results
import pandas as pd

data = {
    'Matroid': matroids_list,
    'invKL': [kl_inverse_fast(m) for m in matroids_list],
    'delform_rhs_qtau': [delform_rhs(m, q_tau, kl_inverse_fast) for m in matroids_list],
    'difference': [x-y for x, y in zip(data['invKL'], data['delform_rhs_tau'])],
    'KL-M/e': [(-1) ** m.rank() * m.contract(m.flats(1)[0]).simplify().lattice_of_flats().kazhdan_lusztig_polynomial() for m in matroids_list],
}

df = pd.DataFrame(data)
display(df)

TypeError: unsupported operand parent(s) for +: 'Univariate Polynomial Ring in q over Rational Field' and 'Fraction Field of Univariate Polynomial Ring in t over Rational Field'